# Exploração Inicial

A QuantumFinance realizou uma parceria com algumas empresas clientes para fornecer o score de crédito de seus clientes optantes para permitir melhores condições de pagamento.
Para que o modelo seja mais simples e diferente do modelo de score principal do banco, será necessário treinar esse modelo com os dados das transações mais recentes dos clientes.

Para permitir governança interna e integração com outros sistemas, esta solução precisa incluir:

Template de repositório para organização dos arquivos (dataset, notebook, modelo, etc.)
Rastreamento dos experimentos do treinamento do modelo
Versionamento do modelo
Disponibilização de endpoint de API seguro com autenticação e throttling
Documentação da API

Afim de validar e tornar como um exemplo de implementação para os parceiros integre uma aplicação modelo no Streamlit com a API disponibilizada.

Dataset https://www.kaggle.com/datasets/parisrohan/credit-score-classification

In [19]:

import pandas as pd
import numpy as np
import re
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("../data/raw/train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

/tmp/ipykernel_212790/3200885717.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/train.csv")


In [3]:
df = df.drop(columns=['ID','Customer_ID','Name','Month','SSN','Interest_Rate','Type_of_Loan','Payment_of_Min_Amount','Payment_Behaviour','Changed_Credit_Limit'])
df.head()

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.843333,3,4,4,3,7,4.0,_,809.98,26.822620,22 Years and 1 Months,49.574949,80.41529543900253,312.49408867943663,Good
1,23,Scientist,19114.12,NaN,3,4,4,-1,NaN,4.0,Good,809.98,31.944960,NaN,49.574949,118.28022162236736,284.62916249607184,Good
2,-500,Scientist,19114.12,NaN,3,4,4,3,7,4.0,Good,809.98,28.609352,22 Years and 3 Months,49.574949,81.699521264648,331.2098628537912,Good
3,23,Scientist,19114.12,NaN,3,4,4,5,4,4.0,Good,809.98,31.377862,22 Years and 4 Months,49.574949,199.4580743910713,223.45130972736786,Good
4,23,Scientist,19114.12,1824.843333,3,4,4,6,NaN,4.0,Good,809.98,24.797347,22 Years and 5 Months,49.574949,41.420153086217326,341.48923103222177,Good


In [4]:
df = df.dropna(axis=0, how='any')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66548 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       66548 non-null  object 
 1   Occupation                66548 non-null  object 
 2   Annual_Income             66548 non-null  object 
 3   Monthly_Inhand_Salary     66548 non-null  float64
 4   Num_Bank_Accounts         66548 non-null  int64  
 5   Num_Credit_Card           66548 non-null  int64  
 6   Num_of_Loan               66548 non-null  object 
 7   Delay_from_due_date       66548 non-null  int64  
 8   Num_of_Delayed_Payment    66548 non-null  object 
 9   Num_Credit_Inquiries      66548 non-null  float64
 10  Credit_Mix                66548 non-null  object 
 11  Outstanding_Debt          66548 non-null  object 
 12  Credit_Utilization_Ratio  66548 non-null  float64
 13  Credit_History_Age        66548 non-null  object 
 14  Total_EMI_p

## Limpeza dos dados

In [6]:
def limpar_e_converter_colunas(df: pd.DataFrame, colunas: list) -> pd.DataFrame:
    """
    Remove caracteres não numéricos (mantendo . e ,) e converte colunas para float.
    Se ocorrer erro de conversão, informa a coluna e os valores problemáticos.
    
    Args:
        df (pd.DataFrame): DataFrame de entrada.
        colunas (list): Lista com os nomes das colunas que serão processadas.
    
    Returns:
        pd.DataFrame: DataFrame com as colunas convertidas para float.
    """
    df = df.copy()  # Evita modificar o original
    
    for coluna in colunas:
        if coluna in df.columns:
            # Remove caracteres não numéricos (mantendo . e ,)
            df[coluna] = df[coluna].astype(str).apply(lambda x: re.sub(r"[^0-9.,]", "", x))
            
            # Substitui vírgula por ponto
            df[coluna] = df[coluna].str.replace(",", ".", regex=False)
            
            try:
                df[coluna] = df[coluna].astype(float)
            except ValueError as e:
                # Identifica valores que causaram erro
                invalidos = df[df[coluna].str.strip() == ''][coluna].tolist()
                raise ValueError(
                    f"Erro ao converter a coluna '{coluna}' para float. "
                    f"Valores inválidos encontrados: {invalidos}"
                ) from e
        else:
            print(f"A coluna '{coluna}' não existe no DataFrame.")
    
    return df

In [7]:
df.head()

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.843333,3,4,4,3,7,4.0,_,809.98,26.822620,22 Years and 1 Months,49.574949,80.41529543900253,312.49408867943663,Good
6,23,Scientist,19114.12,1824.843333,3,4,4,3,8_,4.0,Good,809.98,22.537593,22 Years and 7 Months,49.574949,178.3440674122349,244.5653167062043,Good
8,28_,_______,34847.84,3037.986667,2,4,1,3,4,2.0,Good,605.03,24.464031,26 Years and 7 Months,18.816215,104.291825168246,470.69062692529184,Standard
9,28,Teacher,34847.84,3037.986667,2,4,1,7,1,2.0,Good,605.03,38.550848,26 Years and 8 Months,18.816215,40.39123782853101,484.5912142650067,Good
10,28,Teacher,34847.84_,3037.986667,2,1385,1,3,-1,2.0,_,605.03,33.224951,26 Years and 9 Months,18.816215,58.51597569589465,466.46647639764313,Standard


In [8]:
df = limpar_e_converter_colunas(df, ['Age', 'Annual_Income','Monthly_Inhand_Salary','Num_Bank_Accounts','Num_Credit_Card','Num_of_Loan','Delay_from_due_date','Num_Credit_Inquiries','Outstanding_Debt','Credit_Utilization_Ratio','Total_EMI_per_month','Amount_invested_monthly','Monthly_Balance','Num_of_Delayed_Payment'])

In [9]:
def limpeza_dados(df):
    # --------------------------------------------
    # Substituir idade negativa por NaN (Not a Number) para ser tratada depois
    print("\n--- Corrigindo idade negativa... ---")
    df['Age'] = (
        df['Age']
        .astype(str)                              # Garantir que está como string
        .apply(lambda x: re.sub(r'[^0-9-]', '', x)) # Manter apenas números e sinais de negativo
        .apply(lambda x: int(x) if x not in ['', '-'] else 0) # Converter para int (vazios viram 0)
    )
    df.loc[df['Age'] <= 0, 'Age'] = np.nan

    # Substituir o valor '_' na coluna Credit_Mix por NaN
    print("--- Corrigindo valor '_' em Credit_Mix... ---")
    df['Credit_Mix'] = df['Credit_Mix'].replace('_', np.nan)


    # 1.2 Converter colunas de texto para numérico
    # --------------------------------------------
    # A coluna 'Amount_invested_monthly' e 'Monthly_Balance' estão como objeto (texto)
    print("\n--- Convertendo colunas de texto para numérico... ---")
    df['Amount_invested_monthly'] = pd.to_numeric(df['Amount_invested_monthly'], errors='coerce')
    df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')


    # 1.3 Engenharia na feature 'Credit_History_Age'
    # ----------------------------------------------
    # Extrair anos e meses para uma única coluna numérica (total de meses)
    print("--- Transformando 'Credit_History_Age' em uma feature numérica... ---")

    # Extrai os números de anos e meses usando expressão regular
    history_age_extracted = df['Credit_History_Age'].str.extract(r'(\d+)\s*Years.*?(\d+)\s*Months')
    history_age_extracted.columns = ['Years', 'Months']

    # Converte para numérico
    history_age_extracted['Years'] = pd.to_numeric(history_age_extracted['Years'])
    history_age_extracted['Months'] = pd.to_numeric(history_age_extracted['Months'])

    # Calcula o total de meses e preenche a nova coluna no DataFrame
    df['Credit_History_Total_Months'] = (history_age_extracted['Years'] * 12) + history_age_extracted['Months']

    # Remove a coluna original de texto
    df = df.drop(columns=['Credit_History_Age'])


    # 1.4 Preencher valores ausentes (Imputação)
    # -------------------------------------------
    print("\n--- Preenchendo valores ausentes (NaNs)... ---")

    # Separar colunas numéricas e categóricas
    numeric_cols = df.select_dtypes(include=np.number).columns
    categorical_cols = df.select_dtypes(include=['object']).columns

    # Estratégia para numéricos: preencher com a mediana (mais robusta a outliers)
    numeric_imputer = SimpleImputer(strategy='median')
    df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

    # Estratégia para categóricos: preencher com o valor mais frequente
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])


    print("\n--- Verificação Pós-Limpeza (Não deve haver valores nulos) ---")
    print(df.isnull().sum())
    return df;

In [10]:
df =limpeza_dados(df)
df.head()


--- Corrigindo idade negativa... ---
--- Corrigindo valor '_' em Credit_Mix... ---

--- Convertendo colunas de texto para numérico... ---
--- Transformando 'Credit_History_Age' em uma feature numérica... ---

--- Preenchendo valores ausentes (NaNs)... ---

--- Verificação Pós-Limpeza (Não deve haver valores nulos) ---
Age                            0
Occupation                     0
Annual_Income                  0
Monthly_Inhand_Salary          0
Num_Bank_Accounts              0
Num_Credit_Card                0
Num_of_Loan                    0
Delay_from_due_date            0
Num_of_Delayed_Payment         0
Num_Credit_Inquiries           0
Credit_Mix                     0
Outstanding_Debt               0
Credit_Utilization_Ratio       0
Total_EMI_per_month            0
Amount_invested_monthly        0
Monthly_Balance                0
Credit_Score                   0
Credit_History_Total_Months    0
dtype: int64


,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,Credit_History_Total_Months
0,230.0,Scientist,19114.12,1824.843333,3.0,4.0,4.0,3.0,7.0,4.0,Standard,809.98,26.822620,49.574949,80.415295,312.494089,Good,265.0
6,230.0,Scientist,19114.12,1824.843333,3.0,4.0,4.0,3.0,8.0,4.0,Good,809.98,22.537593,49.574949,178.344067,244.565317,Good,271.0
8,280.0,_______,34847.84,3037.986667,2.0,4.0,1.0,3.0,4.0,2.0,Good,605.03,24.464031,18.816215,104.291825,470.690627,Standard,319.0
9,280.0,Teacher,34847.84,3037.986667,2.0,4.0,1.0,7.0,1.0,2.0,Good,605.03,38.550848,18.816215,40.391238,484.591214,Good,320.0
10,280.0,Teacher,34847.84,3037.986667,2.0,1385.0,1.0,3.0,1.0,2.0,Standard,605.03,33.224951,18.816215,58.515976,466.466476,Standard,321.0


In [11]:
df['Credit_Mix'].value_counts()

Credit_Mix
Standard    37845
Good        16138
Bad         12565
Name: count, dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66548 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          66548 non-null  float64
 1   Occupation                   66548 non-null  object 
 2   Annual_Income                66548 non-null  float64
 3   Monthly_Inhand_Salary        66548 non-null  float64
 4   Num_Bank_Accounts            66548 non-null  float64
 5   Num_Credit_Card              66548 non-null  float64
 6   Num_of_Loan                  66548 non-null  float64
 7   Delay_from_due_date          66548 non-null  float64
 8   Num_of_Delayed_Payment       66548 non-null  float64
 9   Num_Credit_Inquiries         66548 non-null  float64
 10  Credit_Mix                   66548 non-null  object 
 11  Outstanding_Debt             66548 non-null  float64
 12  Credit_Utilization_Ratio     66548 non-null  float64
 13  Total_EMI_per_month  

In [13]:
def tranformando_dados(df):
    cols_to_onehot = [ 'Occupation']
    print(f"\n--- Aplicando One-Hot Encoding nas colunas: {cols_to_onehot} ---")

    # `handle_unknown='ignore'` evita erros se o modelo encontrar uma categoria nova no futuro
    # `sparse_output=False` retorna um array denso (mais fácil de visualizar)
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoded_features = ohe.fit_transform(df[cols_to_onehot])

    # Criar um novo DataFrame com as colunas codificadas
    df_encoded = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(cols_to_onehot))

    # Juntar o DataFrame original com o codificado
    df = pd.concat([df.drop(columns=cols_to_onehot), df_encoded], axis=1)


    # 2.2 Codificação de Variáveis Categóricas Ordinais (Ordinal Encoding)
    # --------------------------------------------------------------------
    # Usamos para colunas onde a ordem importa, como 'Credit_Mix' e a variável alvo 'Credit_Score'.
    print("\n--- Aplicando Ordinal Encoding em 'Credit_Mix'")

    # Definir a ordem explícita das categorias
    credit_mix_order = [ 'Standard', 'Good','Bad']

    # Instanciar o codificador com as categorias definidas
    # ordinal_encoder = OrdinalEncoder(categories=[credit_mix_order])
    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    # Aplicar a codificação
    cols_to_ordinal = ['Credit_Mix']
    
    df[cols_to_ordinal] = ordinal_encoder.fit_transform(df[cols_to_ordinal].fillna('Missing'))


    # ==============================================================================
    # RESULTADO FINAL
    # ==============================================================================
    print("\n\n--- DataFrame Final (Pronto para Machine Learning) ---")
    print(df.head())

    print("\n--- Informações Finais do DataFrame (todas as colunas devem ser numéricas) ---")
    df.info()
    return df

In [14]:
df = tranformando_dados(df)
df.head()


--- Aplicando One-Hot Encoding nas colunas: ['Occupation'] ---

--- Aplicando Ordinal Encoding em 'Credit_Mix'


--- DataFrame Final (Pronto para Machine Learning) ---
      Age  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0   230.0       19114.12            1824.843333                3.0   
6   230.0       19114.12            1824.843333                3.0   
8   280.0       34847.84            3037.986667                2.0   
9   280.0       34847.84            3037.986667                2.0   
10  280.0       34847.84            3037.986667                2.0   

    Num_Credit_Card  Num_of_Loan  Delay_from_due_date  Num_of_Delayed_Payment  \
0               4.0          4.0                  3.0                     7.0   
6               4.0          4.0                  3.0                     8.0   
8               4.0          1.0                  3.0                     4.0   
9               4.0          1.0                  7.0                     1.0   
10   

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,...,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Occupation________
0,230.0,19114.12,1824.843333,3.0,4.0,4.0,3.0,7.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,230.0,19114.12,1824.843333,3.0,4.0,4.0,3.0,8.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,280.0,34847.84,3037.986667,2.0,4.0,1.0,3.0,4.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,280.0,34847.84,3037.986667,2.0,4.0,1.0,7.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,280.0,34847.84,3037.986667,2.0,1385.0,1.0,3.0,1.0,2.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88753 entries, 0 to 66543
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          66548 non-null  float64
 1   Annual_Income                66548 non-null  float64
 2   Monthly_Inhand_Salary        66548 non-null  float64
 3   Num_Bank_Accounts            66548 non-null  float64
 4   Num_Credit_Card              66548 non-null  float64
 5   Num_of_Loan                  66548 non-null  float64
 6   Delay_from_due_date          66548 non-null  float64
 7   Num_of_Delayed_Payment       66548 non-null  float64
 8   Num_Credit_Inquiries         66548 non-null  float64
 9   Credit_Mix                   88753 non-null  float64
 10  Outstanding_Debt             66548 non-null  float64
 11  Credit_Utilization_Ratio     66548 non-null  float64
 12  Total_EMI_per_month          66548 non-null  float64
 13  Amount_invested_month

In [16]:

df_sem_nulos = df.dropna(axis=0, how='any')
df_sem_nulos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44343 entries, 0 to 66547
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          44343 non-null  float64
 1   Annual_Income                44343 non-null  float64
 2   Monthly_Inhand_Salary        44343 non-null  float64
 3   Num_Bank_Accounts            44343 non-null  float64
 4   Num_Credit_Card              44343 non-null  float64
 5   Num_of_Loan                  44343 non-null  float64
 6   Delay_from_due_date          44343 non-null  float64
 7   Num_of_Delayed_Payment       44343 non-null  float64
 8   Num_Credit_Inquiries         44343 non-null  float64
 9   Credit_Mix                   44343 non-null  float64
 10  Outstanding_Debt             44343 non-null  float64
 11  Credit_Utilization_Ratio     44343 non-null  float64
 12  Total_EMI_per_month          44343 non-null  float64
 13  Amount_invested_month

In [20]:
scaler = MinMaxScaler()


colunas_para_normalizar = df.select_dtypes(include='number').columns.difference(['Credit_Score'])


# Aplicar o scaler
df_sem_nulos[colunas_para_normalizar] = scaler.fit_transform(df_sem_nulos[colunas_para_normalizar])

/tmp/ipykernel_212790/2676301493.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sem_nulos[colunas_para_normalizar] = scaler.fit_transform(df_sem_nulos[colunas_para_normalizar])


In [21]:
df_sem_nulos.head()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,...,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Occupation________
0,0.001038,0.000501,0.101127,0.001669,0.002668,0.002694,0.044776,0.001592,0.001542,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.001038,0.000501,0.101127,0.001669,0.002668,0.002694,0.044776,0.001819,0.001542,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.001615,0.001151,0.182628,0.001112,0.002668,0.000673,0.044776,0.000910,0.000771,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.001615,0.001151,0.182628,0.001112,0.002668,0.000673,0.104478,0.000227,0.000771,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.001615,0.001151,0.182628,0.001112,0.923949,0.000673,0.044776,0.000227,0.000771,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
df_sem_nulos.to_csv("../data/processed/train_processed.csv", index=False)